# Bi-LSTM on SST-2

## Librairies

In [ ]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0

In [2]:
import os
import sys

In [3]:
import time
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorWithPadding

from esntorch.utils.embedding import EmbeddingModel

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Global variables

In [5]:
MODEL_FILE = '~/Results/Bi-LSTM/Bi-LSTM/sst-2_bilstm-128_10ep_model.pt'
RESULTS_FILE = '~/Results/Bi-LSTM/Bi-LSTM/sst-2_bilstm-128_10ep_results.pkl'
CACHE_DIR = 'cache_dir/' # put your path here

## Dataset

In [6]:
# rename correct column as 'labels': depends on the dataset you load

def load_and_enrich_dataset(*dataset_name, split, cache_dir):
    
    dataset = load_dataset(*dataset_name, split=split, cache_dir=CACHE_DIR)
    
    def clean(example):
        example['text'] = example['text'].replace('-LRB-', '(').replace('-RRB-', ')').replace(r'\/', r'/')
        return example
        
    if 'sentence' in dataset.column_names:
        dataset = dataset.rename_column('sentence', 'text')
        
    dataset = dataset.map(clean)
    dataset = dataset.rename_column('label', 'labels')
    
    dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding=False), batched=True)
    
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    def add_lengths(sample):
        sample["lengths"] = sum(sample["input_ids"] != 0)
        return sample
    
    dataset = dataset.map(add_lengths, batched=False)
    
    return dataset

In [ ]:
CACHE_DIR = 'cache_dir/' # put your path here

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

train_dataset = load_and_enrich_dataset('glue', 'sst2', split='train', cache_dir=CACHE_DIR).sort("lengths")
val_dataset = load_and_enrich_dataset('glue', 'sst2', split='validation', cache_dir=CACHE_DIR).sort("lengths")
test_dataset = load_and_enrich_dataset('gpt3mix/sst2', split='test', cache_dir=CACHE_DIR).sort("lengths")
def revert(example):
    example['labels'] = np.abs(example['labels'] - 1) # revert labels of test set
    return example
test_dataset = test_dataset.map(revert) # revert labels of test set


dataset_d = {
    'full_train': train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

In [8]:
dataset_d

{'full_train': Dataset({
     features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 67349
 }),
 'train': Dataset({
     features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 67349
 }),
 'val': Dataset({
     features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 872
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 1821
 })}

## Model

In [9]:
class RNN(nn.Module):
    
    def __init__(self, embedding_dim,
                 hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        super().__init__()
        
        self.embedding = EmbeddingModel('bert-base-uncased', device)
        
        # recurrent hidden layers (LSTM)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        # fully connected output layer (fc)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, batch):
        
        packed_embedded = self.dropout(self.embedding.get_embedding(batch))        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)      
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [12]:
output_dim = len(set(dataset_d['full_train']['labels'].tolist()))

In [13]:
output_dim

2

In [14]:
# Parameters

# INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 768
HIDDEN_DIM = 128 # 256 was default
OUTPUT_DIM = output_dim # 1 (binary) or output_dim (*** multiclass ***)
N_LAYERS = 1 # 2 was default
BIDIRECTIONAL = True
DROPOUT = 0.5
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [15]:
# Model

model = RNN(EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [16]:
def count_parameters(model):
    
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 920,066 trainable parameters


## Training

In [17]:
# Adam optimizer
optimizer = optim.Adam(model.parameters())

In [18]:
# Criterion
# criterion = nn.BCEWithLogitsLoss() # binary
criterion = nn.CrossEntropyLoss() # *** multiclass ***

In [19]:
# Put model and criterion to device
model = model.to(device)
criterion = criterion.to(device)

In [20]:
# binary accuracy
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds)) # thresholding output neuron
    correct = (rounded_preds == y).float()            # convert into float for division 
    acc = correct.sum() / len(correct)
    
    return acc

In [21]:
# *** multiclass ***
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    
    return correct.sum().item() / torch.FloatTensor([y.shape[0]])

In [22]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions_raw = model(batch)
                        
        predictions = predictions_raw.squeeze(1)
                
        loss = criterion(predictions, batch['labels'])
        
        # acc = binary_accuracy(predictions, batch.label)        # *** binary *** 
        acc = categorical_accuracy(predictions, batch['labels']) # *** multiclass ***
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch).squeeze(1)
            
            loss = criterion(predictions, batch['labels'])
            
            # acc = binary_accuracy(predictions, batch.label)        # *** binary ***
            acc = categorical_accuracy(predictions, batch['labels']) # *** multiclass ***

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
import time

def epoch_time(start_time, end_time):
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 10 # 30

best_valid_loss = float('inf')

t0 = time.time()

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, dataloader_d['train'], optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, dataloader_d['val'], criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_FILE)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

t1 = time.time()
train_time = t1 - t0
print('training time: {}'.format(train_time))

Epoch: 01 | Epoch Time: 2m 1s
	Train Loss: 0.352 | Train Acc: 84.34%
	 Val. Loss: 0.441 |  Val. Acc: 85.04%
Epoch: 02 | Epoch Time: 1m 59s
	Train Loss: 0.300 | Train Acc: 87.13%
	 Val. Loss: 0.653 |  Val. Acc: 81.05%
Epoch: 03 | Epoch Time: 1m 59s
	Train Loss: 0.267 | Train Acc: 88.74%
	 Val. Loss: 0.620 |  Val. Acc: 79.51%
Epoch: 04 | Epoch Time: 2m 0s
	Train Loss: 0.244 | Train Acc: 89.89%
	 Val. Loss: 0.490 |  Val. Acc: 85.91%
Epoch: 05 | Epoch Time: 2m 0s
	Train Loss: 0.221 | Train Acc: 91.01%
	 Val. Loss: 0.491 |  Val. Acc: 86.50%
Epoch: 06 | Epoch Time: 1m 58s
	Train Loss: 0.205 | Train Acc: 91.73%
	 Val. Loss: 0.607 |  Val. Acc: 81.58%
Epoch: 07 | Epoch Time: 2m 1s
	Train Loss: 0.192 | Train Acc: 92.21%
	 Val. Loss: 0.706 |  Val. Acc: 81.93%
Epoch: 08 | Epoch Time: 2m 0s
	Train Loss: 0.180 | Train Acc: 92.87%
	 Val. Loss: 0.494 |  Val. Acc: 86.10%
Epoch: 09 | Epoch Time: 1m 59s
	Train Loss: 0.178 | Train Acc: 92.85%
	 Val. Loss: 0.496 |  Val. Acc: 85.38%
Epoch: 10 | Epoch Time: 

In [26]:
model = RNN(EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)
model.load_state_dict(torch.load(MODEL_FILE))
model.eval().to(device)

RNN(
  (rnn): LSTM(768, 128, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [27]:
test_loss, test_acc = evaluate(model, dataloader_d['test'], criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

results = test_acc, train_time, count_parameters(model)

Test Loss: 0.385 | Test Acc: 86.27%


In [28]:
results

(0.8627256155014038, 1201.8757109642029, 920066)

In [29]:
with open(RESULTS_FILE, 'wb') as fh:
    pickle.dump(results, fh)